In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score,f1_score
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor


# 이슬점 계산하는 함수
def dew_tem(tem,hum):
    #hum은 0~1 사이값
    import numpy as np
    a=18.678
    b=234.5
    c=257.14
    part1= a*b*c+tem**2-b*(c+tem)*np.log(hum)
    part2= 4*c*(c+tem)*((a*b-tem)*tem+b*(c+tem)*np.log(hum))
    total= (part1-(part1**2-part2)**0.5)/(2*(c+tem))
    return total

## preprocessing
- 기본 공장데이터에 해면기압 추가

In [277]:
main=pd.read_csv("train/plant11_train.csv",encoding='utf-8')
main

,mea_ddhr,tem_in_loc,hum_in_loc,tem_coil_loc,tem_out_loc,hum_out_loc,cond_loc1,cond_24hr,cond_48hr,tem_in_loc_24hr,tem_in_loc_48hr,hum_in_loc_24hr,hum_in_loc_48hr,tem_coil_loc_24hr,tem_coil_loc_48hr
0,2016-04-01 00:00:00,16.00,24.00,11.00,9.00,42.00,0,0,0,15.0,17.0,41.0,54.0,12.0,14.0
1,2016-04-01 03:00:00,14.00,28.00,10.00,7.00,59.00,0,0,0,14.0,16.0,41.0,58.0,10.0,12.0
2,2016-04-01 06:00:00,13.00,33.00,10.00,6.00,56.00,0,0,0,13.0,15.0,38.0,60.0,11.0,13.0
3,2016-04-01 09:00:00,13.00,33.00,10.00,18.00,30.00,0,0,0,14.0,15.0,41.0,64.0,11.0,13.0
4,2016-04-01 12:00:00,16.00,28.00,10.00,18.00,20.00,0,0,0,17.0,16.1,35.0,61.0,13.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56995,2019-03-29 23:10:00,12.99,47.37,12.58,8.23,60.78,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
56996,2019-03-29 23:20:00,13.08,48.02,12.69,8.17,62.61,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
56997,2019-03-29 23:30:00,13.17,48.63,12.66,8.14,62.42,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
56998,2019-03-29 23:40:00,13.08,48.87,12.73,7.92,61.17,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
main=main.drop(main[main.tem_coil_loc_24hr.isnull()].index)

In [43]:
main=main.reset_index(drop=True)

In [45]:
date=pd.read_csv("train_data/plant1_date.csv",encoding='utf-8')

In [165]:
pred=pd.read_csv("train_data/plant_pred.csv",encoding='utf-8')

In [153]:
# aws=pd.read_csv("train_data/plant_637.csv",encoding='utf-8')
# aws.columns
# aws=aws.drop(['aws_loc', 'aws_tem', 'aws_min_rain', 'aws_rain_YN',
#        'aws_wind_d', 'aws_wind_s', 'aws_hum',
#        'aws_sun', 'aws_sun_time'],axis=1)

In [197]:
hpa=pd.read_csv("plant_hpa.csv",encoding='euc-kr',index_col=0)
hpa['해면기압(hPa)']
hpa=hpa.loc[:,['mea_ddhr','해면기압(hPa)']]

In [198]:
new_main=main.merge(hpa).merge(date).merge(pred)

In [199]:
X=new_main.drop(['tem_in_loc_24hr',
       'tem_in_loc_48hr', 'hum_in_loc_24hr', 'hum_in_loc_48hr',
       'tem_coil_loc_24hr', 'tem_coil_loc_48hr','mea_ddhr', 'cond_loc1', 'cond_24hr', 'cond_48hr'],axis=1)
y=new_main.loc[:,['tem_in_loc_24hr',
       'tem_in_loc_48hr', 'hum_in_loc_24hr', 'hum_in_loc_48hr',
       'tem_coil_loc_24hr', 'tem_coil_loc_48hr']]

## Modle_split
- predict temp_24hr , hum_24hr, tem_coil_24hr
- 시계열 자료이므로 random split이 아닌 시간 기준으로 split

In [351]:
train_X=X.iloc[10000:]
test_X=X.iloc[:10000]

In [352]:
train_y=y.iloc[10000:]
test_y=y.iloc[:10000]
train_y

,tem_in_loc_24hr,tem_in_loc_48hr,hum_in_loc_24hr,hum_in_loc_48hr,tem_coil_loc_24hr,tem_coil_loc_48hr
10000,-0.32,-0.04,40.90,34.19,-3.71,-1.65
10001,-0.53,0.14,41.94,30.49,-3.68,-1.50
10002,-0.62,0.14,42.19,29.06,-3.64,-1.50
10003,-0.71,0.14,41.76,28.14,-3.60,-1.50
10004,-0.16,0.26,34.86,29.85,-3.33,-1.35
...,...,...,...,...,...,...
55997,13.42,12.99,39.90,47.37,12.85,12.58
55998,13.33,13.08,41.97,48.02,12.85,12.69
55999,13.30,13.17,44.08,48.63,12.77,12.66
56000,13.27,13.08,44.93,48.87,12.73,12.73


In [364]:
train_y.iloc[:,0]

10000    -0.32
10001    -0.53
10002    -0.62
10003    -0.71
10004    -0.16
         ...  
55997    13.42
55998    13.33
55999    13.30
56000    13.27
56001    13.36
Name: tem_in_loc_24hr, Length: 46002, dtype: float64

## 학습

In [362]:
# model0=RandomForestRegressor()
# model2=RandomForestRegressor()
# model4=RandomForestRegressor()
# model0=LGBMRegressor()
# model2=LGBMRegressor()
# model4=LGBMRegressor()
# model0=LinearRegression()
# model2=LinearRegression()
# model4=LinearRegression()
model0=MLPRegressor()
model2=MLPRegressor()
model4=MLPRegressor()




model0.fit(train_X.to_numpy(),train_y.iloc[:,0])
model2.fit(train_X.to_numpy(),train_y.iloc[:,2])

model4.fit(train_X.to_numpy(),train_y.iloc[:,4])

MLPRegressor()

## 모델 평가

In [ ]:
pred0=model0.predict(test_X)
pred2=model2.predict(test_X)
pred4=model4.predict(test_X)


In [359]:
r2_score(test_y.iloc[:,0],pred0),r2_score(test_y.iloc[:,2],pred2),r2_score(test_y.iloc[:,4],pred4)

(0.9739948115378599, 0.5402486713623653, 0.9878285227846937)

## 예측기준으로 이슬점 계산후 dataframe 만들기

In [229]:
result=pd.DataFrame({"p_tem":pred0,'p_hum':pred2,'p_coil':pred4,'cond_24':new_main.cond_24hr.iloc[:10000]})
result['real_hum']=new_main.hum_in_loc_24hr.iloc[:10000]

In [230]:
r_m=result.p_hum.mean()
r_s=result.p_hum.std()

In [231]:
m=new_main.hum_in_loc.mean()
s=new_main.hum_in_loc.std()

In [232]:
new_hum=[]
for i in result.p_hum:
    new_hum.append(((i-r_m)/r_s)*s+m)

In [233]:
result['new_hum']=new_hum

In [260]:
dew=[]
dew_tf=[]

for i in range(len(result)):
    d = dew_tem(result.p_tem[i],result.new_hum[i]/100)
    dew.append(d)
    dew_tf.append(d>result.p_coil[i])

In [261]:
result['dew_tf']=dew_tf

In [262]:
result['dew']=dew

In [ ]:
result['real_hum']=new_main.hum_in_loc_24hr.iloc[:10000]

## 예상된 값과 실측치 비교
- 결로와 습도의 연관성 높음
- 결로와 표면온도-내부온도의 연관성 높음(파생변수 추가)

In [287]:
# k : coil 표면온도와 실내온도의 차이
k=[]
for i in range(len(main)):
    k.append(main.tem_in_loc[i]-main.tem_coil_loc[i])

In [289]:
main['k']=k

In [291]:
main.k.mean()

1.0762142105264072

In [292]:
main.loc[main.cond_loc1==1].k.mean()

3.5991575091575094

In [294]:
main.loc[main.cond_loc1==1].loc[main.hum_in_loc<70].k.mean()

5.119375

In [304]:
len(main.loc[main.k>5].loc[main.cond_24hr==1]),len(main.loc[main.k>5])

(8, 49)

In [305]:
len(main.loc[main.cond_24hr==1])

273

In [314]:
len(main.loc[main.k>5].loc[main.cond_loc1==1]),len(main.loc[main.k>5])

(19, 49)

## 각 변수의 상관계수 

In [316]:
#
np.correlate(main['cond_loc1'],main['k'])

array([982.57])

In [320]:
np.corrcoef(main['cond_loc1'],main['k'])

array([[1.        , 0.14808304],
       [0.14808304, 1.        ]])

In [325]:
main.corr()

,tem_in_loc,hum_in_loc,tem_coil_loc,tem_out_loc,hum_out_loc,cond_loc1,cond_24hr,cond_48hr,tem_in_loc_24hr,tem_in_loc_48hr,hum_in_loc_24hr,hum_in_loc_48hr,tem_coil_loc_24hr,tem_coil_loc_48hr,k
tem_in_loc,1.000000,0.443123,0.992952,0.975117,0.229139,-0.025558,-0.040257,-0.052517,0.986364,0.976316,0.425700,0.413155,0.990901,0.984101,0.004416
hum_in_loc,0.443123,1.000000,0.443059,0.448727,0.811902,0.166720,0.035560,0.010457,0.441122,0.436685,0.527496,0.360447,0.458311,0.457521,-0.023857
tem_coil_loc,0.992952,0.443059,1.000000,0.958858,0.253936,-0.042942,-0.054005,-0.058685,0.982679,0.975276,0.434013,0.427656,0.995306,0.988828,-0.114136
tem_out_loc,0.975117,0.448727,0.958858,1.000000,0.124133,-0.002887,-0.015542,-0.043869,0.960120,0.948429,0.420059,0.397221,0.959468,0.952367,0.083499
hum_out_loc,0.229139,0.811902,0.253936,0.124133,1.000000,0.096530,-0.015416,0.002317,0.236545,0.236373,0.412542,0.273213,0.263561,0.265015,-0.221836
cond_loc1,-0.025558,0.166720,-0.042942,-0.002887,0.096530,1.000000,-0.004813,-0.001038,-0.044779,-0.053383,0.019778,-0.030327,-0.044077,-0.052132,0.148083
cond_24hr,-0.040257,0.035560,-0.054005,-0.015542,-0.015416,-0.004813,1.000000,-0.004759,-0.025717,-0.045050,0.167783,0.019950,-0.043571,-0.044046,0.118209
cond_48hr,-0.052517,0.010457,-0.058685,-0.043869,0.002317,-0.001038,-0.004759,1.000000,-0.039848,-0.025333,0.034332,0.164660,-0.053730,-0.042721,0.054929
tem_in_loc_24hr,0.986364,0.441122,0.982679,0.960120,0.236545,-0.044779,-0.025717,-0.039848,1.000000,0.986495,0.444258,0.426494,0.993014,0.990989,-0.022618
tem_in_loc_48hr,0.976316,0.436685,0.975276,0.948429,0.236373,-0.053383,-0.045050,-0.025333,0.986495,1.000000,0.442695,0.444786,0.982825,0.993043,-0.041337


In [327]:
t_e=[]
c_e=[]
for i in range(10000):
    t_e.append(result.p_tem[i]-main.tem_in_loc_24hr[i])
    c_e.append(result.p_coil[i]-main.tem_coil_loc_24hr[i])

In [330]:
result['te']=t_e
result['ce']=c_e

In [333]:
result.te.mean(),result.ce.mean(),result.te.std(),result.ce.std()

(-0.6286165647835943,
 -0.7085157047739793,
 3.0906031768886884,
 2.7154271805549386)

,tem_in_loc,hum_in_loc,tem_coil_loc,tem_out_loc,hum_out_loc,해면기압(hPa),year,month,day,hour,week_1,week_2,week_3,week_4,week_5,week_6,week_7,week_8,week_9,week_10,week_11,week_12,week_13,week_14,week_15,week_16,week_17,week_18,week_19,week_20,week_21,week_22,week_23,week_24,week_25,week_26,week_27,week_28,week_29,week_30,week_31,week_32,week_33,week_34,week_35,week_36,week_37,week_38,week_39,week_40,week_41,week_42,week_43,week_44,week_45,week_46,week_47,week_48,week_49,week_50,week_51,week_52,tem_24hr,hum_24hr,rain_p_24hr,wind_s_24hr,wind_d_24hr,tem_48hr,hum_48hr,rain_p_48hr,wind_s_48hr,wind_d_48hr,sky_1_24hr,sky_2_24hr,sky_3_24hr,sky_4_24hr,sky_1_48hr,sky_2_48hr,sky_3_48hr,sky_4_48hr,rain_t_0_24hr,rain_t_1_24hr,rain_t_2_24hr,rain_t_3_24hr,rain_t_0_48hr,rain_t_1_48hr,rain_t_2_48hr,rain_t_3_48hr
10000,-4.50,29.09,-6.23,-13.57,48.99,1031.600000,2018,1,27,6,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-2.000000,45.000000,20.000000,2.600000,331.000000,-5.000000,45.000000,20.000000,4.000000,332.000000,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0
10001,-4.96,31.41,-6.31,-13.11,50.64,1031.400000,2018,1,27,7,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-2.666667,48.333333,20.000000,3.000000,330.000000,-5.666667,48.333333,20.000000,4.366667,328.333333,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0
10002,-5.29,33.67,-6.38,-13.69,56.87,1031.700000,2018,1,27,8,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-3.333333,51.666667,20.000000,3.400000,329.000000,-6.333333,51.666667,20.000000,4.733333,324.666667,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0
10003,-5.35,36.94,-6.19,-9.32,43.71,1031.900000,2018,1,27,9,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-4.000000,55.000000,20.000000,3.800000,328.000000,-7.000000,55.000000,20.000000,5.100000,321.000000,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0
10004,-4.87,37.33,-6.08,-5.75,33.27,1032.200000,2018,1,27,10,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-4.333333,55.000000,20.000000,4.066667,328.666667,-7.000000,55.000000,20.000000,5.033333,324.666667,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55997,14.49,59.77,12.85,8.60,78.17,1014.083333,2019,3,27,23,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.888889,57.777778,26.388889,3.563889,109.888889,5.888889,68.750000,23.611111,1.597222,183.638889,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0
55998,14.55,58.73,12.88,8.32,79.24,1014.166667,2019,3,27,23,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.111111,57.222222,26.111111,3.561111,118.111111,6.111111,67.500000,23.888889,1.627778,186.611111,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0
55999,14.46,58.30,12.85,8.08,80.22,1014.250000,2019,3,27,23,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.333333,56.666667,25.833333,3.558333,126.333333,6.333333,66.250000,24.166667,1.658333,189.583333,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0
56000,14.43,58.39,12.85,8.08,81.17,1014.333333,2019,3,27,23,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.555556,56.111111,25.555556,3.555556,134.555556,6.555556,65.000000,24.444444,1.688889,192.555556,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0


In [368]:
result.loc[result.new_hum>80]

,p_tem,p_hum,p_coil,cond_24,real_hum,new_hum,dew_tf,dew,te,ce
248,20.427902,74.547175,17.191146,0,75.00,81.440072,False,17.144358,-0.572098,-1.808854
249,20.629813,75.983708,17.893170,1,80.00,83.364597,False,17.711131,-0.370187,-0.106830
250,18.613860,74.500312,16.015281,0,82.00,81.377290,False,15.363658,-2.386140,-2.984719
3986,21.050792,74.262928,20.005373,0,64.47,81.059266,False,17.677248,4.120792,3.535373


In [370]:
new_main.columns

Index(['mea_ddhr', 'tem_in_loc', 'hum_in_loc', 'tem_coil_loc', 'tem_out_loc',
       'hum_out_loc', 'cond_loc1', 'cond_24hr', 'cond_48hr', 'tem_in_loc_24hr',
       'tem_in_loc_48hr', 'hum_in_loc_24hr', 'hum_in_loc_48hr',
       'tem_coil_loc_24hr', 'tem_coil_loc_48hr', '해면기압(hPa)', 'year', 'month',
       'day', 'hour', 'week_1', 'week_2', 'week_3', 'week_4', 'week_5',
       'week_6', 'week_7', 'week_8', 'week_9', 'week_10', 'week_11', 'week_12',
       'week_13', 'week_14', 'week_15', 'week_16', 'week_17', 'week_18',
       'week_19', 'week_20', 'week_21', 'week_22', 'week_23', 'week_24',
       'week_25', 'week_26', 'week_27', 'week_28', 'week_29', 'week_30',
       'week_31', 'week_32', 'week_33', 'week_34', 'week_35', 'week_36',
       'week_37', 'week_38', 'week_39', 'week_40', 'week_41', 'week_42',
       'week_43', 'week_44', 'week_45', 'week_46', 'week_47', 'week_48',
       'week_49', 'week_50', 'week_51', 'week_52', 'tem_24hr', 'hum_24hr',
       'rain_p_24hr', 'wind_s_24hr

In [377]:
aws=pd.read_csv('train_data/plant_616.csv',encoding='utf-8')

In [378]:
main.merge(aws).corr()

,tem_in_loc,hum_in_loc,tem_coil_loc,tem_out_loc,hum_out_loc,cond_loc1,cond_24hr,cond_48hr,tem_in_loc_24hr,tem_in_loc_48hr,hum_in_loc_24hr,hum_in_loc_48hr,tem_coil_loc_24hr,tem_coil_loc_48hr,k,aws_loc,aws_tem,aws_min_rain,aws_rain_YN,aws_wind_d,aws_wind_s,aws_spot_pr,aws_sea_pr,aws_hum,aws_sun,aws_sun_time
tem_in_loc,1.000000,0.442818,0.992967,0.975149,0.227766,-0.025665,-0.040390,-0.052670,0.986341,0.976297,0.425471,0.412810,0.990892,0.984084,0.007447,NaN,0.968960,0.028169,0.045941,-0.105781,-0.096965,NaN,NaN,0.297809,NaN,NaN
hum_in_loc,0.442818,1.000000,0.442782,0.448459,0.811638,0.167055,0.035661,0.010512,0.440699,0.435890,0.527137,0.359925,0.457530,0.456736,-0.022696,NaN,0.456463,0.084146,0.305147,-0.154823,-0.167520,NaN,NaN,0.749194,NaN,NaN
tem_coil_loc,0.992967,0.442782,1.000000,0.958957,0.252494,-0.043120,-0.054206,-0.058895,0.982683,0.975254,0.433773,0.427305,0.995302,0.988816,-0.110995,NaN,0.946544,0.029377,0.048824,-0.122664,-0.117897,NaN,NaN,0.332883,NaN,NaN
tem_out_loc,0.975149,0.448459,0.958957,1.000000,0.122884,-0.002927,-0.015598,-0.043962,0.960149,0.948402,0.419802,0.396876,0.959434,0.952331,0.086093,NaN,0.986395,0.023450,0.032786,-0.085507,-0.032007,NaN,NaN,0.218543,NaN,NaN
hum_out_loc,0.227766,0.811638,0.252494,0.122884,1.000000,0.096688,-0.015454,0.002311,0.235042,0.235399,0.412190,0.272662,0.262629,0.264074,-0.220685,NaN,0.150010,0.085508,0.313743,-0.189023,-0.304009,NaN,NaN,0.835779,NaN,NaN
cond_loc1,-0.025665,0.167055,-0.043120,-0.002927,0.096688,1.000000,-0.004828,-0.001053,-0.044918,-0.053365,0.019837,-0.030312,-0.044053,-0.052113,0.148766,NaN,0.003327,0.003981,0.079086,0.013745,0.035606,NaN,NaN,0.059276,NaN,NaN
cond_24hr,-0.040390,0.035661,-0.054206,-0.015598,-0.015454,-0.004828,1.000000,-0.004774,-0.025821,-0.045023,0.167956,0.019993,-0.043546,-0.044019,0.118797,NaN,-0.009017,0.003981,0.003635,-0.032805,-0.006622,NaN,NaN,-0.021368,NaN,NaN
cond_48hr,-0.052670,0.010512,-0.058895,-0.043962,0.002311,-0.001053,-0.004774,1.000000,-0.039977,-0.025288,0.034402,0.164784,-0.053716,-0.042693,0.055313,NaN,-0.040407,-0.004077,-0.005492,-0.029334,-0.018403,NaN,NaN,-0.003705,NaN,NaN
tem_in_loc_24hr,0.986341,0.440699,0.982683,0.960149,0.235042,-0.044918,-0.025821,-0.039977,1.000000,0.986480,0.444073,0.426182,0.993010,0.990981,-0.019726,NaN,0.956185,0.026490,0.035500,-0.122370,-0.112686,NaN,NaN,0.297519,NaN,NaN
tem_in_loc_48hr,0.976297,0.435890,0.975254,0.948402,0.235399,-0.053365,-0.045023,-0.025288,0.986480,1.000000,0.442513,0.444477,0.982811,0.993038,-0.041300,NaN,0.943499,0.027381,0.039644,-0.110023,-0.100014,NaN,NaN,0.296738,NaN,NaN


In [379]:
asos=pd.read_csv('train_data/plant_asos.csv',encoding='utf-8')

In [380]:
main.merge(asos).corr()

,tem_in_loc,hum_in_loc,tem_coil_loc,tem_out_loc,hum_out_loc,cond_loc1,cond_24hr,cond_48hr,tem_in_loc_24hr,tem_in_loc_48hr,hum_in_loc_24hr,hum_in_loc_48hr,tem_coil_loc_24hr,tem_coil_loc_48hr,k,asos_tem,asos_wind,asos_wind_dir,asos_hum,asos_steam,asos_dew,asos_spot_pr,asos_sea_pr,asos_tem_grd
tem_in_loc,1.000000,0.446297,0.992913,0.974564,0.232451,-0.024951,-0.038597,-0.051358,0.986272,0.976413,0.429113,0.414383,0.990720,0.983992,0.007409,0.964157,0.097625,-0.025573,0.016468,-0.824181,-0.824818,0.212736,0.289160,0.143446
hum_in_loc,0.446297,1.000000,0.447159,0.450993,0.812495,0.165514,0.033875,0.007663,0.443986,0.439215,0.528775,0.359511,0.461489,0.460537,-0.030562,0.449014,0.274280,-0.055405,-0.096586,-0.540113,-0.540034,0.712005,-0.229528,-0.336888
tem_coil_loc,0.992913,0.447159,1.000000,0.958175,0.258019,-0.042079,-0.052855,-0.057599,0.982745,0.975610,0.438291,0.429567,0.995347,0.989080,-0.111483,0.941387,0.098857,-0.051897,-0.014129,-0.817274,-0.817415,0.252229,0.237699,0.098302
tem_out_loc,0.974564,0.450993,0.958175,1.000000,0.125564,-0.002463,-0.013232,-0.042570,0.959424,0.947821,0.423143,0.397720,0.958510,0.951460,0.087005,0.982921,0.094923,0.033208,0.091585,-0.810124,-0.811424,0.127902,0.277915,0.139852
hum_out_loc,0.232451,0.812495,0.258019,0.125564,1.000000,0.096769,-0.018871,-0.001326,0.239597,0.240019,0.413151,0.273154,0.267899,0.269242,-0.227281,0.139732,0.233639,-0.197287,-0.290951,-0.349522,-0.347745,0.837480,-0.263813,-0.348695
cond_loc1,-0.024951,0.165514,-0.042079,-0.002463,0.096769,1.000000,-0.004694,-0.000777,-0.044542,-0.053806,0.016738,-0.035800,-0.043448,-0.052159,0.145421,0.002393,0.045941,0.039148,0.042766,-0.028641,-0.029268,0.067302,-0.042271,-0.048014
cond_24hr,-0.038597,0.033875,-0.052855,-0.013232,-0.018871,-0.004694,1.000000,-0.004649,-0.025147,-0.044693,0.166474,0.016888,-0.042551,-0.043447,0.121986,-0.004637,-0.010438,-0.016802,-0.005593,0.015664,0.015150,-0.028910,-0.004868,-0.001890
cond_48hr,-0.051358,0.007663,-0.057599,-0.042570,-0.001326,-0.000777,-0.004649,1.000000,-0.038400,-0.024824,0.031814,0.163196,-0.052541,-0.041910,0.055195,-0.037428,-0.012290,-0.015591,-0.011189,0.047800,0.047965,-0.009152,0.002075,0.009400
tem_in_loc_24hr,0.986272,0.443986,0.982745,0.959424,0.239597,-0.044542,-0.025147,-0.038400,1.000000,0.986420,0.447212,0.427105,0.992934,0.990803,-0.021158,0.951326,0.108092,-0.036621,0.001575,-0.810289,-0.810705,0.217028,0.291276,0.146194
tem_in_loc_48hr,0.976413,0.439215,0.975610,0.947821,0.240019,-0.053806,-0.044693,-0.024824,0.986420,1.000000,0.445344,0.445066,0.982858,0.992952,-0.043562,0.939262,0.110622,-0.026310,0.012961,-0.801024,-0.801213,0.217285,0.291083,0.145771
